# Use the twitter API to source tweets with a given set of hashtags

In [1]:
import tweepy

## Instantiate the twitter API and get the country code for the UK

- The user needs to have a developer account and add their own keys
- The UK is actually the 3rd country in the list returned by `search_geo`

In [2]:
MAX_TWEETS = 50

bearer_token = "xxxxxx"
CONSUMER_TOKEN = "xxxxxx"
CONSUMER_SECRET = "xxxxxx"

auth = tweepy.OAuthHandler(CONSUMER_TOKEN, CONSUMER_SECRET)
api = tweepy.API(auth)


places = api.search_geo(query="United Kingdom",granularity="country")
place_id = places[2].id

Unauthorized: 401 Unauthorized
32 - Could not authenticate you

## Instantiate the BERT tokenizers and model

- My jupyter lab crashes when I use the tokenizer
- Will probably need a GPU for reasonable runtime

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## Build the query

- Add desired hashtags to the `hashtags` list
- Current form builds an `x OR y OR z` query, avoids retweets, and gets tweets from the UK

In [ ]:
hashtags = ['Skype']#'Teams', 'Zoom', 

n_hashtags = len(hashtags)
hashtag_string = ''
for hashtag_index, hashtag in enumerate(hashtags):
    if hashtag_index == n_hashtags - 1:
        
        hashtag_string += f'#{hashtag} '
    else:
        hashtag_string += f'#{hashtag} OR '
        
        
query = f'{hashtag_string}   place:{place_id} -filter:retweets'
print(query)

## Use the tweepy cursor and `search_tweets` to retrieve some tweets

- Builds a list of the raw text from tweets 
- ### not kosher for ethics - same cell is below with attempts to get sentiment on tweets without local storage of text

In [ ]:
tweets = []

for tweet in tweepy.Cursor(api.search_tweets, q=query, count=500, tweet_mode='extended').items(MAX_TWEETS):#
    text = tweet.full_text.split('http')[0]
    tweets.append(text)
    print(text)
    print('\n')

In [ ]:
tweet_sentiments = []

for tweet in tweepy.Cursor(api.search_tweets, q=query, count=100, tweet_mode='extended').items(MAX_TWEETS):

    text = tweet.full_text.split('http')[0]
    try:
        text_tokens = tokenizer.encode(text)
        sentiment = model.predict(text_tokens)

        tweet_sentiments.append(sentiment)
        print(sentiment)
        print('\n')
    except:
        print(f"Tweet failed: {text}")
        pass